In [ ]:
%ls

In [ ]:
!mkdir '.\NeCTIS Model Data'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
import fontstyle
import os

In [ ]:
def raw_to_clean(line):
    line = re.sub('( {1,})',' ',line)
    line = re.sub('-$','',line) #to remove the - in the end
    line = re.sub('<','',line)
    line = re.sub('-',' ',line)
    line = re.sub('(>\w+)','',line)
    line = re.sub(' $','',line) #to remove the space in the end
    line = re.sub('^ ','',line) #to remove the space in the beginning
    line = re.sub('( {1,})',' ',line)
    return line

In [ ]:
def Conversion(infile,outfile):
    df = pd.read_csv(infile)
    # #Uncomment df_new files to store data in dataframes
    with open(outfile,'w') as w:
        lines = df['Clean']    #Clean line
        if 'Finegrain' in outfile:
            raw_lines = df['Raw_Tagged']    #raw line for fine grain data
        elif 'Coarse' in outfile:
            raw_lines = df['Coarse_tag']    #raw line for Coarse data
        # df_new = pd.DataFrame(columns= ['Word Index','Word','Comp Length','_','Related word index','Relation Tag'])
        z = 0
        for k in range(df.shape[0]):
            print(k+1)
            raw_line = raw_lines[k].strip()
            line = lines[k]
            comps_and_words = raw_line.strip().split() #space separated raw line gives list of compounds and individual words
            clean_tokens = line.strip().split() #space separated clean line gives clean tokens
            outmost_comp_list = [token for token in comps_and_words if '<' in token] #identifying compounds from comps_and_words
            c = 0
            d = 0
#             print(raw_line)
    #         w.write(raw_line+'\n'*2)
            i = 0
    #         df_new.loc[z] = [raw_line,'','','','','']
            z += 1
            while i < len(clean_tokens):   ###traversing through sentence tokens
                if c< len(outmost_comp_list):
                    outcomp = outmost_comp_list[c]
                clean_outcomp = raw_to_clean(outcomp) ### clean tokens of the outermost/parent compound
                token = comps_and_words[d] ### choosing token i.e., compound and word from comps_and_words list
                word = clean_tokens[i]
                if word == token: ###for individual words other than compounds
#                     print('\033[94m'+f'{i+1}\t{word}\tCompNo\t_\t{len(clean_tokens)+1}\tNo_rel'+'\033[0m')### Red line
                    w.write(f'{i+1}\t{word}\tCompNo\t_\t{len(clean_tokens)+1}\tNo_rel'+'\n')
    #                 df_new.loc[z] = [i+1,word,'CompNo','_',len(clean_tokens)+1,'No_rel']
                    z += 1
                    i += 1
                    d += 1
                else:
                    rem_string = outcomp
                    comp_len = len(clean_outcomp.split())
                    for p in range(comp_len):
                        subword = clean_outcomp.split()[p] ### getting p-th subword of the compound
                        if p==comp_len-1:   ###for last subword of an out-most compound
#                             print('\033[92m'+f'{i+1}\t{subword}\tComp{comp_len}\t_\t{len(clean_tokens)+1}\tComp_root'+'\033[0m')### Green line
                            w.write(f'{i+1}\t{subword}\tComp{comp_len}\t_\t{len(clean_tokens)+1}\tComp_root'+'\n')
    #                         df_new.loc[z] = [i+1,subword,'Comp'+str(comp_len),'_',len(clean_tokens)+1,'Comp_root']
                            z += 1
                            i += 1
                        else:    ### for remaining subwords in compound
                            subword_end = re.search('[^>]'+subword,rem_string).end() #getting end of the subword
                            # using ^> so that words like ta, di don't interfere with tags
                            rem_string = rem_string[subword_end:] ### remaining string after the subword/token
                            n = 0
                            ind = 0
                            p = 0
                            if rem_string[0] == '>': ###if remaining string starts with tag >\w+
                                flag = 1
                            else:                   
                                flag = 0
                            for ind in range(len(rem_string)):
                                if rem_string[ind] == '<': ###adding 1 to n if encountered an open bracket
                                    n -= 1
                                elif rem_string[ind] == '>': ###subtracting 1 from n if encountered an open bracket
                                    n += 1
                                elif rem_string[ind] == '-': ###adding 1 if encountered hyphen
                                    p += 1
                                if n-flag == 0: ### setting p to 0 if a compound is completed i.e., n-flag==0
                                    p = 0
                                if rem_string[0] == '>': #case1: remaining string starts with tag => >\w+
                                    #subcase1: remaining string has no new compound/ has an immediate relation word i.e., -\w+
                                    if rem_string.count('<')==0 or len(re.findall('^>\w+-\w+',rem_string))>0:
                                        st = ind+re.search('-\w+>',rem_string).end()-1
                                        temp = rem_string[st:]
                                        tag = re.findall('>(\w+)',temp)[0]
                                        break
                                    else:
                                        if len(re.findall('^>\w+>\w+',rem_string))==0: #subcase2 counterpart
                                            if n-flag == 1 and p>=0:
                                                st = ind
                                                temp = rem_string[st:]
                                                tag = re.findall('>(\w+)',temp)[0]
                                                break
                                        else: #subcase2: multiple tags closing
                                            #subsubcase1: multiple tags closing after the relation word
                                            if len(re.findall('-\w+(?:>\w+)+>(\w+)',rem_string))>0:  
                                                st = re.search('-\w+(?:>\w+)+>(\w+)',rem_string).end()-1
                                                tag = re.findall('-\w+(?:>\w+)+>(\w+)',rem_string)[0]
                                            else: #subsubcase2: normal tags after relation word
                                                g = re.findall('^(?:>\w+)*>\w+',rem_string)[0].count('>')
                                                if n-flag-g==1:
                                                    st = ind
                                                    temp = rem_string[st:]
                                                    tag = re.findall('>(\w+)',temp)[0]
                                            break
                                #remaining cases: remaining string starting with related word => -\w+ (or) compound => -<\w+-\w+>\w+
                                else: 
                                    if n-flag == 1:
                                        st = ind
                                        temp = rem_string[st:]
                                        tag = re.findall('>(\w+)',temp)[0]
                                        break
                            pre_tag_string = rem_string[:st]
                            num = len(raw_to_clean(pre_tag_string).split())
#                             print('\033[91m'+f'{i+1}\t{subword}\tComp{comp_len}\t_\t{i+num+1}\t{tag}'+'\033[0m')
                            w.write(f'{i+1}\t{subword}\tComp{comp_len}\t_\t{i+num+1}\t{tag}'+'\n')
    #                         df_new.loc[z] = [i+1,subword,'Comp'+str(comp_len),'_',i+num+1,tag]
                            z += 1
                            i += 1
                    c += 1
                    d += 1
#             print(f'{i+1}\tDUMMY\tCompNo\t_\t{0}\troot')
            w.write(f'{i+1}\tDUMMY\tCompNo\t_\t{0}\troot\n')
    #         df_new.loc[z]= [i+1,'DUMMY','CompNo','_',0,'root']
            z += 2
#             print('\n')
            w.write('\n')

#### With Context files

In [ ]:
%cd "With Context CSV files"

csv_files = ['./train.csv','./dev.csv','./test.csv','./outofDomain.csv']
out_files_Coarse = ['../NeCTIS Model Data/With Context/Coarse/'+csv[2:-3]+'txt' for csv in csv_files]
out_files_finegrain = ['../NeCTIS Model Data/With Context/Finegrain/'+csv[2:-3]+'txt' for csv in csv_files]
outfiles = [out_files_Coarse,out_files_finegrain]
for k in range(2):
    i = 0
    for csv in csv_files:
        outfile = outfiles[k][i]
        infile = csv
        Conversion(infile,outfile)
        i += 1

#### No Context files

In [ ]:
%cd "../No Context CSV files"

csv_files = ['./train.csv','./dev.csv','./test.csv','./outofDomain.csv']
out_files_Coarse = ['../NeCTIS Model Data/Without Context/Coarse/'+csv[2:-3]+'txt' for csv in csv_files]
out_files_finegrain = ['../NeCTIS Model Data/Without Context/Finegrain/'+csv[2:-3]+'txt' for csv in csv_files]
outfiles = [out_files_Coarse,out_files_finegrain]
for k in range(2):
    i = 0
    for csv in csv_files:
        outfile = outfiles[k][i]
        infile = csv
        Conversion(infile,outfile)
        i += 1

In [ ]:
%cd "../"

#### The format1-refers to the morph data and format2-refers to the label data

In [ ]:
def format1(file,outfile):
    with open(file) as f:
        lines = f.readlines()
    with open(outfile,'w') as o:
        for line in lines:
            if line!= '\n':
                lst = line.strip().split('\t')
                lstnew = lst[:3]+[lst[2]]+lst[4:]+[lst[2]]
                line_new = '\t'.join(lstnew)
                o.write(line_new+'\n')
            else:
                o.write(line)

In [ ]:
def format2(file,outfile):
    with open(file) as f:
        lines = f.readlines()
    with open(outfile,'w') as o:
        for line in lines:
            if line!= '\n':
                lst = line.strip().split('\t')
                lstnew = lst[:3]+[lst[2]]+lst[4:]+[lst[5]]
                line_new = '\t'.join(lstnew)
                o.write(line_new+'\n')
            else:
                o.write(line)

In [ ]:
root = '' #root directory of the NeCTIS model data to generate new format data 
lst = []
for path, subdirs, files in os.walk(root):
    for name in files:
        lst.append(os.path.join(path, name))
# lst = [x for x in lst if 'Domain' not in x] #to skip out of domain folder
print('List is:\n')
for p in lst:
    print(p)
infiles = lst
outfiles = [] #create a list of file paths for the NeCTIS model data new format
for i in range(len(infiles)):
    print(infiles[i])
    print(outfiles[i])
    format1(infiles[i],outfiles[i])

In [ ]:
root = '' #root directory of the NeCTIS model data to generate new format data 
lst = []
for path, subdirs, files in os.walk(root):
    for name in files:
        lst.append(os.path.join(path, name))
# lst = [x for x in lst if 'Domain' not in x] #to skip out of domain folder
print('List is:\n')
for p in lst:
    print(p)
infiles = lst
outfiles = [] #create a list of file paths for the NeCTIS model data new format
for i in range(len(infiles)):
    print(infiles[i])
    print(outfiles[i])
    format2(infiles[i],outfiles[i])